##  Least Squares Support Vector Machine (LSSVM) model, optimized by Particle Swarm Optimization (PSO).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from pyswarm import pso  # Particle Swarm Optimization library

# Load train and test data from Excel files
train = pd.read_excel('train.xlsx').values
test = pd.read_excel('test.xlsx').values

# Split into features (X) and targets (y)
p_train = train[:, :-1]  # Features of training data
t_train = train[:, -1]   # Target of training data
p_test = test[:, :-1]    # Features of test data
t_test = test[:, -1]     # Target of test data

# Data normalization using MinMaxScaler (same as MATLAB's mapminmax)
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

pn_train = scaler_x.fit_transform(p_train)  # Normalize training features
tn_train = scaler_y.fit_transform(t_train.reshape(-1, 1)).flatten()  # Normalize training targets
pn_test = scaler_x.transform(p_test)  # Normalize test features
tn_test = scaler_y.transform(t_test.reshape(-1, 1)).flatten()  # Normalize test targets

# Define PSO optimization function for hyperparameter tuning
def pso_svr(params):
    C, gamma = params
    svr_model = SVR(C=C, gamma=gamma, kernel='rbf')
    svr_model.fit(pn_train, tn_train)
    predictions = svr_model.predict(pn_test)
    mse = mean_squared_error(tn_test, predictions)
    return mse

# Set PSO options
lb = [1, 1e-3]  # Lower bounds of C and gamma
ub = [1000, 100]  # Upper bounds of C and gamma

# Optimize hyperparameters using PSO
best_params, _ = pso(pso_svr, lb, ub, swarmsize=20, maxiter=50)
best_c, best_gamma = best_params

# Train the SVR model with optimal hyperparameters
svr_model = SVR(C=best_c, gamma=best_gamma, kernel='rbf')
svr_model.fit(pn_train, tn_train)

# Predict on training and test sets
predict_train = svr_model.predict(pn_train)
predict_test = svr_model.predict(pn_test)

# Reverse normalization
predict_train_original = scaler_y.inverse_transform(predict_train.reshape(-1, 1)).flatten()
predict_test_original = scaler_y.inverse_transform(predict_test.reshape(-1, 1)).flatten()

# Compute RMSE for training and test sets
rmse_train = np.sqrt(mean_squared_error(t_train, predict_train_original))
rmse_test = np.sqrt(mean_squared_error(t_test, predict_test_original))

# Compute R² score for training and test sets
r2_train = r2_score(t_train, predict_train_original)
r2_test = r2_score(t_test, predict_test_original)

print(f"Training RMSE: {rmse_train}")
print(f"Test RMSE: {rmse_test}")
print(f"Training R²: {r2_train}")
print(f"Test R²: {r2_test}")

# Plot true vs predicted values for the test set
plt.figure()
plt.plot(t_test, 'o-r', label='True Values')
plt.plot(predict_test_original, '*-b', label='Predicted Values')
plt.legend()
plt.show()


## 基于粒子群优化 (PSO) 算法优化支持向量机 (SVM) 参数

In [ ]:
pip install scikit-learn pyswarm numpy

In [ ]:
import pandas as pd

# 读取训练和测试数据
train_data = pd.read_csv('train_data.csv')  # 替换为你的数据路径
test_data = pd.read_csv('test_data.csv')  # 替换为你的数据路径

# 提取特征和目标
pn_train = train_data.iloc[:, :-1].values  # 假设最后一列是目标值
tn_train = train_data.iloc[:, -1].values
pn_test = test_data.iloc[:, :-1].values
tn_test = test_data.iloc[:, -1].values


In [ ]:
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# 定义支持向量机的PSO优化过程
def pso_svm(tn_train, pn_train, tn_test, pn_test, pso_option):
    # PSO参数初始化
    Vcmax = pso_option['k'] * pso_option['popcmax']
    Vcmin = -Vcmax
    Vgmax = pso_option['k'] * pso_option['popgmax']
    Vgmin = -Vgmax
    eps = 10**(-4)

    # 初始化种群和速度
    pop = np.zeros((pso_option['sizepop'], 2))
    V = np.zeros((pso_option['sizepop'], 2))
    fitness = np.zeros(pso_option['sizepop'])

    for i in range(pso_option['sizepop']):
        pop[i, 0] = (pso_option['popcmax'] - pso_option['popcmin']) * np.random.rand() + pso_option['popcmin']
        pop[i, 1] = (pso_option['popgmax'] - pso_option['popgmin']) * np.random.rand() + pso_option['popgmin']
        V[i, 0] = Vcmax * np.random.rand()
        V[i, 1] = Vgmax * np.random.rand()

        C = pop[i, 0]
        gamma = pop[i, 1]

        # 训练SVM模型
        model = SVR(C=C, gamma=gamma, kernel='rbf')
        model.fit(pn_train, tn_train)
        predict_test = model.predict(pn_test)

        # 计算适应度值 (RMSE)
        test_RMSE = np.sqrt(mean_squared_error(tn_test, predict_test))
        fitness[i] = test_RMSE

    # 找到最优初始解
    best_fitness = np.min(fitness)
    best_index = np.argmin(fitness)
    local_fitness = fitness.copy()
    global_x = pop[best_index, :]
    local_x = pop.copy()

    # 迭代优化
    for _ in range(pso_option['maxgen']):
        for j in range(pso_option['sizepop']):
            # 更新速度
            V[j, :] = pso_option['wV'] * V[j, :] + \
                      pso_option['c1'] * np.random.rand() * (local_x[j, :] - pop[j, :]) + \
                      pso_option['c2'] * np.random.rand() * (global_x - pop[j, :])

            # 限制速度
            V[j, 0] = np.clip(V[j, 0], Vcmin, Vcmax)
            V[j, 1] = np.clip(V[j, 1], Vgmin, Vgmax)

            # 更新位置
            pop[j, :] = pop[j, :] + pso_option['wP'] * V[j, :]
            pop[j, 0] = np.clip(pop[j, 0], pso_option['popcmin'], pso_option['popcmax'])
            pop[j, 1] = np.clip(pop[j, 1], pso_option['popgmin'], pso_option['popgmax'])

            # 自适应变异
            if np.random.rand() > 0.5:
                k = np.random.choice([0, 1])
                if k == 0:
                    pop[j, k] = (20 - 1) * np.random.rand() + 1
                if k == 1:
                    pop[j, k] = (pso_option['popgmax'] - pso_option['popgmin']) * np.random.rand() + pso_option['popgmin']

            # 重新计算适应度值
            C = pop[j, 0]
            gamma = pop[j, 1]
            model = SVR(C=C, gamma=gamma, kernel='rbf')
            model.fit(pn_train, tn_train)
            predict_test = model.predict(pn_test)

            test_RMSE = np.sqrt(mean_squared_error(tn_test, predict_test))
            fitness[j] = test_RMSE

            # 更新个体最优
            if fitness[j] < local_fitness[j]:
                local_x[j, :] = pop[j, :]
                local_fitness[j] = fitness[j]

            # 更新全局最优
            if fitness[j] < best_fitness:
                global_x = pop[j, :]
                best_fitness = fitness[j]

    best_c = global_x[0]
    best_gamma = global_x[1]

    return best_c, best_gamma

# PSO 参数
pso_option = {
    'c1': 1.5,
    'c2': 1.7,
    'maxgen': 50,
    'sizepop': 20,
    'k': 0.6,
    'wV': 1,
    'wP': 1,
    'popcmax': 1000,
    'popcmin': 1,
    'popgmax': 100,
    'popgmin': 1e-3
}

# 示例数据集（用户应自行加载训练集和测试集）
# pn_train = 训练集特征, tn_train = 训练集目标, pn_test = 测试集特征, tn_test = 测试集目标
# 用户需要用自己的数据替换下述伪数据
pn_train = np.random.rand(100, 6)  # 替换为真实数据
tn_train = np.random.rand(100)  # 替换为真实数据
pn_test = np.random.rand(50, 6)  # 替换为真实数据
tn_test = np.random.rand(50)  # 替换为真实数据

# PSO优化支持向量机参数
best_c, best_gamma = pso_svm(tn_train, pn_train, tn_test, pn_test, pso_option)

# 输出最佳参数
print(f'最佳C值: {best_c}, 最佳gamma值: {best_gamma}')
